In [21]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import re
from PIL import Image


In [3]:
data = 'data-99spots/'
files = [f for f in listdir(data) if isfile(join(data, f)) and f.endswith('.csv')]

# for file in files:
#     with open(data+files[1], 'rb') as csvfile:
#         reader = csv.reader(csvfile, delimiter=',', quotechar='|')
#         for row in reader:
#             print ', '.join(row)

csv_file = data + files[1]
df = pd.read_csv(csv_file)
# saved_column = df.column_name #you can also use df['column_name']
# print df['Cell_ID']
# a =df[:3]
xy = np.concatenate((np.reshape(df.Cell_X, (-1,1)), np.reshape(df.Cell_Y, (-1,1))),axis=1)
features = {}
# biomarker = []

def determineCohort(spot_name):
    cohort_1 = ['000', '005', '026', '031', '046', '055', '060', '081', '086']
    cohort_2 = ['001', '006', '011', '016', '025', '030', '043', '045', '056',
         '061', '066', '076', '080', '085', '096']
    cohort_3 = ['002', '007', '012', '017', '020', '024', '029', '032', '034',
        '036', '039', '044', '049', '052', '057', '062', '067', '071', '072',
        '079', '084', '089', '091', '095']
    cohort_4 = ['003', '008', '013', '015', '018', '021', '023', '028', '033',
        '038', '041', '048', '053', '058', '063', '065', '068', '070', '073',
        '078', '083', '088', '090', '093']
    if spot_name in cohort_1:
        return 1
    elif spot_name in cohort_2:
        return 2
    elif spot_name in cohort_3:
        return 3
    elif spot_name in cohort_4:
        return 4
    else:
        print 'Spot does not belong to any cohorts!'
        
for idx, col in enumerate(list(df)):
    
    pass
    
# #      m = re.search('^[^_]+(?=_)', col)
#     biomarker.append(tuple(col.split('_')[0:2]))
#     features[biomarker[idx]] = df[col].values
        
     
     # print m.group(0)
biomarker, location, col = zip(*[(col.split('_')[0], col.split('_')[1], col) for col in list(df) if col.split('_')[0] != 'Cell'])
# b, l = [col.spl]
# biomarker = [m.group(0) for col in list(df) for m in [re.search('^[^_]+(?=_)', col)] if col != 'Cell']
uniq_biomarker = list(set(biomarker))
uniq_location = list(set(location))

k = 0
for i, bio in enumerate(uniq_biomarker):
    features[bio] ={}
    for j, loc in enumerate(uniq_location):
#         features[bio][loc] = df[col[i*len(uniq_location) + j]].values
        features[bio][loc] = df[col[k]].values
#         print i*(len(uniq_location)) + j
        k+=1
        
#         print i*len(uniq_location) + j
        
# for i in range(len(col)):
#     features[biomarker[i]][location[i]] = df[col[i]]
    
        


# biomarker[:] = list(set((bio for bio in biomarker if bio != 'Cell')))

# for idx, col in enumerate(list(df)):
#     print idx, col
    

# filename = '011_Quant.csv'
# spot_name = re.search('\d{2,3}', filename).group(0)





In [79]:
print biomarker[4]
print location[2]
print col[2]
print uniq_biomarker
# print uniq_location

Her2
Mem
EGFR_Mem_Quant
['PR', 'Her2', 'PCAD', 'EGFR', 'ER']


In [37]:
for i in range(5):
    print features.keys()[i]
    print features[features.keys()[i]].keys()

    

PR
['Mem', 'Nuc', 'Cyt']
Her2
['Mem', 'Nuc', 'Cyt']
PCAD
['Mem', 'Nuc', 'Cyt']
EGFR
['Mem', 'Nuc', 'Cyt']
ER
['Mem', 'Nuc', 'Cyt']


In [73]:
print biomarker[2]
spot_num = '011'

EGFR


In [74]:
image_dir = 'images'
a=[join(image_dir,f) for f in listdir(image_dir) if uniq_biomarker[0] in f]

In [75]:
image = [join(a[0],f) for f in listdir(a[0]) if spot_num in f]
print image

['images/PR-allTissue/PR_AFRemoved_011.tif']


In [76]:
im = Image.open(image[0])

In [77]:
im.show()

In [1]:
import numpy
import PIL
 
# Convert Image to array
img = PIL.Image.open("foo.jpg").convert("L")
arr = numpy.array(img)
 
# Convert array to Image
img = PIL.Image.fromarray(arr)

AttributeError: 'module' object has no attribute 'Image'